In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk


  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-ib250efp/gym-walk_1b7dcd0ef6904bfe89aeb29fce253f4a
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-ib250efp/gym-walk_1b7dcd0ef6904bfe89aeb29fce253f4a
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4053 sha256=8abaaf9976cf3d87da101fa14ab3ca516c2e302f0da7807f5e9d92f2e102c54c
  Stored in directory: /tmp/pip-ephem-wheel-cache-7g7ag2hy/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [ ]:


import warnings ; warnings.filterwarnings('ignore')
import gym, gym_walk
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)



In [ ]:


def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

env = gym.make('SlipperyWalkFive-v0')
P = env.env.P
init_state = env.reset()
goal_state = 6
LEFT, RIGHT = range(2)
P
init_state

state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:2 - reward:0.0 - done:False - info:{'prob': 0.16666666666666666}


In [ ]:
pi_1 = lambda s: {
    0:LEFT, 1:LEFT, 2:LEFT, 3:LEFT, 4:LEFT, 5:LEFT, 6:LEFT
}[s]
print_policy(pi_1, P, action_symbols=('<', '>'), n_cols=7)

print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_1, goal_state=goal_state)*100,
    mean_return(env, pi_1)))
print("HARSHAVARDHAN")

Policy:
|           | 01      < | 02      < | 03      < | 04      < | 05      < |           |
Reaches goal 3.00%. Obtains an average undiscounted return of 0.0300.
HARSHAVARDHAN


In [ ]:
pi_2= lambda s:{
    0:LEFT,1:RIGHT,2:LEFT,3:LEFT,4:RIGHT,5:RIGHT,6:LEFT
}[s]

print_policy(pi_2, P, action_symbols=('<', '>'), n_cols=7)
print("HARSHAVARDHAN")

Policy:
|           | 01      > | 02      < | 03      < | 04      > | 05      > |           |
HARSHAVARDHAN


In [ ]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
      V=np.zeros(len(P),dtype=np.float64)
      for s in range(len(P)):
        for prob, next_state, reward, done in P[s][pi(s)]:
          V[s]+=prob*(reward+gamma*prev_V[next_state]*(not done))
      if np.max(np.abs(prev_V-V))<theta:
        break
      prev_V=V.copy()
    return V

In [ ]:
V1 = policy_evaluation(pi_1, P)
print(V1)
print_state_value_function(V1, P, n_cols=7, prec=5)
print("HARSHAVARDHAN")

[0.         0.00274725 0.01098901 0.03571429 0.10989011 0.33241758
 0.        ]
State-value function:
|           | 01 0.00275 | 02 0.01099 | 03 0.03571 | 04 0.10989 | 05 0.33242 |           |
HARSHAVARDHAN


In [ ]:
V2 = policy_evaluation(pi_2, P)
print(V2)
print_state_value_function(V2, P, n_cols=7, prec=5)
print("HARSHAVARDHAN")

[0.   0.15 0.2  0.35 0.8  0.95 0.  ]
State-value function:
|           | 01   0.15 | 02    0.2 | 03   0.35 | 04    0.8 | 05   0.95 |           |
HARSHAVARDHAN


In [ ]:
print(V1>=V2)
if(np.sum(V1>=V2)==7):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)==7):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")
print("HARSHAVARDHAN")

[ True False False False False False  True]
The second policy is the better policy
HARSHAVARDHAN


In [ ]:
from itertools import count

In [ ]:
def decay_schedule(init_value,min_value,decay_ratio,max_steps,log_start=-2,log_base=10):
    decay_steps=int(max_steps * decay_ratio)
    rem_steps=max_steps - decay_steps
    values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
    values = (values - values.min()) / (values.max() - values.min())
    values = (init_value - min_value) * values + min_value
    values = np.pad(values, (0,rem_steps), 'edge')
    return values

In [ ]:
def generate_trajectory(pi,env,max_steps=20):
  done,trajectory = False,[]
  while not done:
    state = env.reset()
    for t in count():
      action = pi(state)
      next_state,reward,done,_ = env.step(action)
      experience = (state,action,reward,next_state,done)
      trajectory.append(experience)
      if done:
        break
      if t>=max_steps -1:
        trajectory=[]
        break
      state=next_state
  return np.array(trajectory,np.object)

In [ ]:
def mc_prediction(env, gamma=1.0, init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
               init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
               n_episodes=3000, max_steps=200, first_visit=True):


    nS, nA = env.observation_space.n, env.action_space.n


    discount = np.logspace(0, max_steps, num=max_steps, base=gamma, endpoint=False)
    alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
    Q = np.zeros(nS)
    Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float64)
    for e in tqdm(range(n_episodes), leave=False):
        # Generate a trajectory
        traj = gen_traj(pi,env, max_steps)
        visited = np.zeros((nS, nA), dtype=np.bool)

        for t, (state, _i, reward, _, _) in enumerate(traj):
            if visited[state] and first_visit:
                continue
            visited[state] = True

            n_steps = len(traj[t:])
            G = np.sum(discount[:n_steps] * traj[t:, 2])
            Q[state] = Q[state] + alphas[e] * (G - Q[state])

        Q_track[e] = Q
    return Q.copy(),Q_track